Gonna see what I can do with the Quijote sims...

In [7]:
%matplotlib inline
from matplotlib import pyplot as plt

In [8]:
import numpy as np
from os import path 
import h5py

In [9]:
quijote_dir = '/scratch/users/swmclau2/UatuQuijote/'

In [10]:
cosmos = [('fiducial','Box000', (0.3175, 0.834) ),
          ('Om_p','Box001', (0.3275, 0.834) ),
          ('Om_m','Box002', (0.3075, 0.834) ),
          ('s8_p','Box003', (0.3175, 0.849) ),
          ('s8_m','Box004', (0.3175, 0.819) )]

In [11]:
output_fname = path.join(quijote_dir, 'UatuQuijoteTraining.hdf5')

In [12]:
with h5py.File(output_fname, 'w') as output_file:
    output_file.attrs['start'] = 0
    output_file.attrs['stop'] = 5
    output_file.attrs['shape'] = (1600, 128,128)
    output_file.attrs['mean'] = 0.0
    output_file.attrs['std'] = 1.0


for simname, boxno, y, in cosmos:
    print simname
    sublogsims = np.zeros((400*4, 128, 128, 1))
    for i in xrange(400):
        sim = np.load(path.join(quijote_dir, simname, str(i), 'df_z=0.npy'))
        logsim = np.log10(sim-np.min(sim)+1e-2)
        tmp = []
        for subbox_i, subbox_j in [(0,0), (0,1), (1,0), (1,1)]:
            sublogsim = logsim[subbox_i*128:(subbox_i+1)*128, subbox_j*128:(subbox_j+1)*128]
            tmp.append(np.expand_dims(sublogsim, axis = 3))
        sublogsims[i*4:(i+1)*4] = np.array(tmp)
        
    with h5py.File(output_fname) as output_file:
        grp = output_file.create_group(boxno)
        grp.create_dataset('X', data = sublogsims)
        grp.create_dataset('Y', data= np.tile(np.array(y), (sublogsims.shape[0], 1)))

fiducial


/share/software/user/open/py-scipystack/1.0_py27/lib/python2.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


Om_p
Om_m
s8_p
s8_m


In [13]:
output_fname = path.join(quijote_dir, 'UatuQuijoteTest.hdf5')

In [14]:
with h5py.File(output_fname, 'w') as output_file:
    output_file.attrs['start'] = 0
    output_file.attrs['stop'] = 5
    output_file.attrs['shape'] = (400, 128,128)
    output_file.attrs['mean'] = 0.0
    output_file.attrs['std'] = 1.0


for simname, boxno, y, in cosmos:
    print simname
    sublogsims = np.zeros((400*4, 128, 128))
    for i in xrange(400, 500):
        sim = np.load(path.join(quijote_dir, simname, str(i), 'df_z=0.npy'))
        logsim = np.log10(sim-np.min(sim)+1e-2)
        tmp = []
        for subbox_i, subbox_j in [(0,0), (0,1), (1,0), (1,1)]:
            sublogsim = logsim[subbox_i*128:(subbox_i+1)*128, subbox_j*128:(subbox_j+1)*128]
            tmp.append(np.expand_dims(sublogsim, axis = 3))
        sublogsims[(i-400)*4:(i+1-400)*4] = np.array(tmp)
        
    with h5py.File(output_fname) as output_file:
        grp = output_file.create_group(boxno)
        grp.create_dataset('X', data = sublogsims)
        grp.create_dataset('Y', data= np.tile(np.array(y), (sublogsims.shape[0], 1)))

fiducial


/share/software/user/open/py-scipystack/1.0_py27/lib/python2.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


ValueError: could not broadcast input array from shape (4,128,128,1) into shape (4,128,128)

In [ ]:
from uatu.watchers.Dataset import DatasetFromFile

In [ ]:
batch_size = 32
transform = lambda x:x
fname = path.join(quijote_dir, 'UatuQuijoteTraining.hdf5')
train_dset = DatasetFromFile(fname,batch_size, shuffle=True, augment=True, train_test_split = 0.8,\
                                 whiten = True, cache_size = 100, transform=transform)


In [ ]:
for i, (X, Y) in enumerate(train_dset):
    print X.shape
    plt.imshow(X[0])
    plt.title(str(Y[0,0]) + str(Y[0,1]))
    plt.show();
    
    if i==10:
        break

In [ ]:
fname